In [1]:
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
import os, numpy as np, scipy as sp, scipy.io, scipy.io.wavfile
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import random
import pandas as pd

2023-04-30 16:39:46.357296: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-30 16:39:46.378851: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 16:39:46.715267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
path = r'df_complete_data.pkl'
df = pd.read_pickle(path)

In [3]:

#check correctness of dataframe loaded
for i in range(68261):
  if len(df["recording"][i]) != 4096:
    print("problem sir")

In [4]:
new_df = df[['patient_id', 'murmur']]
new_df = new_df.drop_duplicates()

In [5]:
new_df = new_df.reset_index(drop=True)

In [6]:
nb_murmur_present = (new_df.loc[new_df['murmur'] == 1]).shape[0]
nb_murmur_absent = (new_df.loc[new_df['murmur'] == 0]).shape[0]
print(nb_murmur_present)
print(nb_murmur_absent)

179
695


In [7]:
new_df

,patient_id,murmur
0,2530.0,0.0
1,9979.0,1.0
2,13918.0,1.0
3,14241.0,1.0
4,14998.0,0.0
...,...,...
869,85340.0,0.0
870,85341.0,0.0
871,85343.0,1.0
872,85345.0,0.0


In [8]:
list_id = new_df['patient_id']
list_label = new_df['murmur']

In [9]:
def get_balanced_dataset(X,y):
  ones = np.where(np.array(y)==1)
  zeros = np.where(np.array(y)==0)
  ones = ones[0]
  zeros = zeros[0]
  trunc = ones.shape[0] - zeros.shape[0]
  zeros = zeros[:trunc]
  new_X=[]
  new_y=[]
  for i in ones:
    new_X.append(X[i])
    new_y.append(y[i])
  for j in zeros:
    new_X.append(X[j])
    new_y.append(y[j])
  return new_X,new_y

In [10]:
list_id,list_label = get_balanced_dataset(list_id,list_label)

In [11]:
id_train, id_valtest, label_train, label_valtest = train_test_split(list_id, list_label , test_size=0.2, random_state=42)
id_val, id_test, label_val, label_test = train_test_split(id_valtest, label_valtest , test_size=0.5, random_state=42)

In [12]:
def select_rows(id_list,dframe):
  #for id in list_id:
  sub_df = df.loc[df['patient_id'].isin([int(id) for id in id_list])]
  return sub_df

df_train = select_rows(id_train,df)
df_val = select_rows(id_val,df)
df_test = select_rows(id_test,df)

In [13]:
(df_train.loc[df_train['murmur'] == 0]).shape[0]

12317

In [14]:
X_train = np.vstack(df_train['recording'])
y_train = np.array(df_train['murmur'])

X_val = np.vstack(df_val['recording'])
y_val = np.array(df_val['murmur'])

X_test = np.vstack(df_test['recording'])
y_test = np.array(df_test['murmur'])

In [15]:
X_train = X_train.reshape(len(X_train),len(X_train[1]),1)
X_val = X_val.reshape(len(X_val),len(X_val[1]),1)
X_test = X_test.reshape(len(X_test),len(X_test[1]),1)

In [16]:
from tensorflow import keras
from tensorflow.keras import layers

In [17]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [18]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

In [22]:
input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=1,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25
)


In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 4096, 1)]    0           []                               
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 4096, 1)     2           ['input_2[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 multi_head_attention_4 (MultiH  (None, 4096, 1)     7169        ['layer_normalization_8[0][0]',  
 eadAttention)                                                    'layer_normalization_8[0][0]']  
                                                                                            

In [28]:
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["acc"],
)
#model.summary()

callbacks = [tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]


history = model.fit(
    X_train,
    np.array(y_train),
    batch_size=8,
    epochs=15,
    callbacks=callbacks,
    validation_data=(X_val,np.array(y_val)),
    verbose=1,
)

#model.evaluate(x_test, y_test, verbose=1)

Epoch 1/15
2898/2898 [==============================] - 567s 195ms/step - loss: 776.4698 - acc: 0.5037 - val_loss: 478.1349 - val_acc: 0.5253
Epoch 2/15
2898/2898 [==============================] - 565s 195ms/step - loss: 418.7880 - acc: 0.5739 - val_loss: 342.5678 - val_acc: 0.5103
Epoch 3/15
2898/2898 [==============================] - 567s 196ms/step - loss: 147.9356 - acc: 0.6297 - val_loss: 137.5140 - val_acc: 0.5448
Epoch 4/15
2898/2898 [==============================] - 566s 195ms/step - loss: 33.3281 - acc: 0.6460 - val_loss: 68.1872 - val_acc: 0.5007
Epoch 5/15
2898/2898 [==============================] - 565s 195ms/step - loss: 10.1928 - acc: 0.6257 - val_loss: 40.9135 - val_acc: 0.4680
Epoch 6/15
2898/2898 [==============================] - 566s 195ms/step - loss: 7.4538 - acc: 0.6043 - val_loss: 29.3798 - val_acc: 0.4453
Epoch 7/15
2898/2898 [==============================] - 565s 195ms/step - loss: 3.1865 - acc: 0.5909 - val_loss: 21.6828 - val_acc: 0.4412
Epoch 8/15
2898/

In [29]:
score = model.evaluate(X_test, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

2023-04-30 19:15:35.133190: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.00GiB (rounded to 8589934592)requested by op model_1/multi_head_attention_4/einsum/Einsum
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-30 19:15:35.133269: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-04-30 19:15:35.133286: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 232, Chunks in use: 230. 58.0KiB allocated for chunks. 57.5KiB in use in bin. 1.5KiB client-requested in use in bin.
2023-04-30 19:15:35.133296: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 22, Chunks in use: 22. 11.0KiB allocated for chunks. 11.0KiB in use in bin. 11.0KiB client-requested in use in bin.
2023-04-30 19:1

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/multi_head_attention_4/einsum/Einsum' defined at (most recent call last):
    File "/home/ali/Downloads/ENTER/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/ali/Downloads/ENTER/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/ali/Downloads/ENTER/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/ali/Downloads/ENTER/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/ali/Downloads/ENTER/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/ali/Downloads/ENTER/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/al

In [30]:
pred = model.predict(X_test)
print(pred)

2023-04-30 19:15:54.447912: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.00GiB (rounded to 8589934592)requested by op model_1/multi_head_attention_4/einsum/Einsum
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-30 19:15:54.447999: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-04-30 19:15:54.448017: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 232, Chunks in use: 231. 58.0KiB allocated for chunks. 57.8KiB in use in bin. 1.4KiB client-requested in use in bin.
2023-04-30 19:15:54.448028: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 22, Chunks in use: 22. 11.0KiB allocated for chunks. 11.0KiB in use in bin. 11.0KiB client-requested in use in bin.
2023-04-30 19:1

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/multi_head_attention_4/einsum/Einsum' defined at (most recent call last):
    File "/home/ali/Downloads/ENTER/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/ali/Downloads/ENTER/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/ali/Downloads/ENTER/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/ali/Downloads/ENTER/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/ali/Downloads/ENTER/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/ali/Downloads/ENTER/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/al

In [ ]:
def check_max_pred(prd):
  max=0
  for i in prd:
    if i[0] > max:
      max = i[0]
  print(max)

In [ ]:
check_max_pred(pred)

In [ ]:
list_patient_rows = list(df_test.groupby(['patient_id'])['patient_id'].count())

In [ ]:
def cumulative(lists): 
    cu_list = [] 
    length = len(lists) 
    cu_list = [sum(lists[0:x:1]) for x in range(0, length+1)] 
    return cu_list[1:]

In [ ]:
pred_groupby_patient = np.split(pred,cumulative(list_patient_rows))
pred_groupby_patient = pred_groupby_patient[:-1]

df_test = df_test.astype({'murmur':'int'})
y_test_grouped = df_test[['patient_id','murmur']].drop_duplicates()['murmur'].to_list()

In [ ]:
def mean_pred(prediction):
  pred = []
  for i in prediction:
    lst = [ j[0] for j in i]
    avg = sum(lst)/len(lst)
    if avg > 0.5:
      pred.append(1)
    else:
      pred.append(0)
  return pred

def majority_pred(prediction):
  pred = []
  for i in prediction:
    lst = [ j[0] for j in i]
    if (sum(elem > 0.5 for elem in lst)) > (len(lst)/2):
      pred.append(1)
    else:
      pred.append(0)
  return pred


In [ ]:
pred1 = mean_pred(pred_groupby_patient)
pred2 = majority_pred(pred_groupby_patient)



In [ ]:
print(pred1)
print(pred2)
print(y_test_grouped)
print(accuracy_score(pred1 , y_test_grouped))

[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0]
[0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
